In [1]:
import itertools
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from handcrafted.app.dataset.dataset_loader import DatasetLoader
from handcrafted.app.dataset.utils.dataset_creator import DatasetCreator
from handcrafted.app.dataset.utils.frames_splitter import FramesSplitter
from handcrafted.app.model.model_statistics import ModelStatistics

2025-03-31 17:38:56.437684: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 17:38:56.494973: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 17:38:56.756674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-31 17:38:56.756775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-31 17:38:56.814481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
loader = DatasetLoader(directory="./data/frames_no_bg/")

12023it [00:13, 886.45it/s] 


In [3]:
num_signers = -1
if num_signers == -1:
    signers = loader.signers
    num_signers = loader.num_signers
else:
    signers = dict(itertools.islice(loader.signers.items(), num_signers))
splitter = FramesSplitter(signers, val_split=0.3, test_split=0.3, frames_split=0.1, seed=42)

In [6]:
for signer in signers.values():
    signer_frames = []
    for video in signer.videos:
        signer_frames.extend(video.frames)
    print(f"Signer {signer.id} has {len(signer_frames)} frames")

Signer 117 has 2329 frames
Signer 26 has 9778 frames
Signer 63 has 3888 frames
Signer 35 has 5184 frames
Signer 42 has 3297 frames
Signer 48 has 816 frames
Signer 104 has 186 frames
Signer 70 has 1136 frames
Signer 94 has 2662 frames
Signer 77 has 1805 frames
Signer 99 has 1752 frames
Signer 93 has 2772 frames
Signer 45 has 3523 frames
Signer 32 has 10037 frames
Signer 38 has 31192 frames
Signer 103 has 3085 frames
Signer 0 has 17182 frames
Signer 19 has 18262 frames
Signer 13 has 87112 frames
Signer 56 has 12833 frames
Signer 21 has 36376 frames
Signer 52 has 4783 frames
Signer 17 has 433 frames
Signer 60 has 3794 frames
Signer 107 has 505 frames
Signer 41 has 45179 frames
Signer 36 has 15448 frames
Signer 4 has 28130 frames
Signer 97 has 3619 frames
Signer 73 has 413 frames
Signer 90 has 11725 frames
Signer 3 has 1959 frames
Signer 100 has 1671 frames
Signer 31 has 9623 frames
Signer 46 has 2441 frames
Signer 89 has 2673 frames
Signer 10 has 37293 frames
Signer 119 has 530 frames
Sig

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = splitter.split(X_content=lambda f: f)

In [ ]:
# Get the total of different labels
num_classes = num_signers
print(num_classes)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
BATCH_SIZE = 512
dataset_creator = DatasetCreator()
# TODO: add shuffle = True
train_dataset = dataset_creator.create_custom_dataset(X_train, y_train_encoded, batch_size=BATCH_SIZE)
val_dataset = dataset_creator.create_custom_dataset(X_val, y_val_encoded, batch_size=BATCH_SIZE)
test_dataset = dataset_creator.create_custom_dataset(X_test, y_test_encoded, batch_size=BATCH_SIZE)


In [ ]:
xgb_params = {
    "objective":"multi:softmax",
    "num_class":num_classes,
    "eval_metric":"mlogloss",
    "max_depth":6,
    "learning_rate":0.1,
    "n_estimators":100,
    "subsample":0.8,
    "colsample_bytree":0.8,
    "gamma":0,
    "min_child_weight":1,
    "reg_lambda":1,
    "reg_alpha":0,
    "tree_method":"hist", # Change to "gpu_hist" for GPU acceleration
    # "device":"cuda", # Use GPU
}

In [ ]:
xgb_model = None
for train_batch in tqdm(val_dataset):
    X_batch, y_batch = train_batch.load(num_aug=3, shuffle=True)
    dtrain = xgb.DMatrix(X_batch, label=y_batch)
    xgb_model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=10, xgb_model=xgb_model)

In [ ]:
y_pred = []
y_test_full = []
for test_batch in tqdm(test_dataset):
    X_batch, y_batch = test_batch.load(num_aug=0, shuffle=True)
    y_test_full.extend(y_batch)
    dtest = xgb.DMatrix(X_batch)
    y_pred_batch = xgb_model.predict(dtest)
    y_pred.extend(y_pred_batch)

In [ ]:
stats = ModelStatistics(save_name=f"svc_signer_test_{len(y_test_full)}", save_dir="signer/plots")
stats.print_accuracy(y_test_full, y_pred)
stats.plot_confusion_matrix(y_test_full, y_pred, save=True, plot=True)